In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from fake_useragent import UserAgent
ua = UserAgent()
import time

In [2]:
def get_web_page(url,r=None):
    headers = {'user-agent':ua.random,'Accept-Language': 'zh-tw'}
    if r is None:
        resp = requests.get(url=url,headers = headers)
    else:
        resp = requests.get(url=url,headers = headers, cookies={'urlJumpIp':str(r)})
    if resp.status_code != 200:
        print('Invalid url:', resp.url)
        return(None)
    else:
        return(BeautifulSoup(resp.text,'html.parser'))

In [3]:
def detail_page(url,r):
    soup = get_web_page(url,r)
    obj = soup.find_all('ul',attrs={'class':'listInfo clearfix'})
    
    result = pd.DataFrame(columns=['head','price','floor','size','addr','suite','suite_detail','other','link'])
    for oo in obj:
        head = oo.h3.a.contents[0]
        link = 'https:' + oo.h3.a['href']
        tmp1 = oo.find_all('p')
        info1 = tmp1[0].contents
        info1 = [info1[i].split()[0] for i in range(len(info1)) if i%2 == 0]
        #坪/樓層/
        size = np.nan; floor = np.nan; suite_detail = np.nan; other = []
        for info in info1[1:]:
            if '坪' in info:
                size = info
            elif '樓層' in info:
                floor = info[3:]
            elif '房' in info:
                suite_detail = info
            else:
                other = other + [info]
        
        info2 = tmp1[1].em.contents[0]
        price = oo.find('div', attrs={'class':'price'}).i.contents[0]
        price = int(''.join(price.split(',')))
        result = pd.concat([result, 
                            pd.DataFrame([[head, price, floor, size, info2, info1[0], suite_detail, other, link]],
                            columns=['head','price','floor','size','addr','suite','suite_detail','other','link'])],0)
    return(result.reset_index(drop=True))

In [6]:
def crawler_main(url,r):
    soup = get_web_page(url,r)
    ttlnum = int(soup.find('a', attrs={'class':'pageNum-form'})['data-total'])
    ttlpage = int(ttlnum/30)+1
    
    result = pd.DataFrame(columns=['head','price','floor','size','addr','suite','suite_detail','other','link'])
    
    print('Processing Pages ({} Pages In Total): '.format(ttlpage))
    for l in range(ttlpage):
        print('{}'.format(l), end='\t')
        url2 = url + '&firstRow={}'.format(l*30)
        tmp = detail_page(url2,r)
        result = pd.concat([result, tmp], 0)
        time.sleep(1)
    
    print('\nProcess Done!!\n')
    return(result.reset_index(drop=True))

#### Taipei Rent Data

In [ ]:
url = 'https://rent.591.com.tw/?kind=0&region=1'
taipei_rent = crawler_main(url,1)
taipei_rent.to_csv('./taipei.csv',index=False)
#newtaipei_rent = crawler_main(url,3)
#newtaipei_rent.to_csv('./new_taipei.csv',index=False)